In [4]:
from glob import glob
import os.path as op
import nibabel as nib
import shutil
from shutil import copyfile
import pathlib
import pickle
import gzip
from glob import glob
import nibabel as nb
import numpy as np
import os

In [5]:
#Step 1: Define fMRIPrep working directory i.e., the TEMP folder
work_dir = ('/projects/.................')

#Step 2: Define fMRIPrep derivatives directory
deriv_dir = ('/projects/................../derivatives/fmriprep/fmriprep')

#Step 3: Define subject ID
#subID = ["102", "103", "104", "105", "106", "107", "108", "109", "110", "111", "112", "113", "114", "115", "116", "117", "118", "119", "120",
#          "121", "122", "123", "124", "125", "126", "127", "128", "129", "130", "131", "132", "133", "136", "137", "138", "139", "140", "141", "142",
#          "143",	"201", "202", "204", "205", "206", "207", "208", "209", "210", "211", "212", "213", "214", "215", "216", "217", "218", "219",
#          "220", "221", "222", "223", "224", "225", "226", "227", "228", "229", "230", "231", "232", "233", "234", "235", "236", "237", "238", "239",
#          "240",	"241", "242", "243"]
subID = ['101', '238']
for sub in subID:
    sub_in_dir = op.join(work_dir, '{0}/fmriprep_wf/single_subject_{0}_wf'.format(sub))
    task_dirs = glob(op.join(sub_in_dir, 'func_preproc_task_{0}_echo_1_wf'.format("REST")))
    #print(task_dirs)
    
    for task_dir in task_dirs:
        bb_wf_dir = op.join(task_dir, 'bold_bold_trans_wf')
        bf_dirs = sorted(glob(op.join(bb_wf_dir, '_bold_file_*'))) 
        
        for bf_dir in bf_dirs:
            bf_dir_list = bf_dir.split('..')           
            idx = bf_dir_list.index('sub-{0}'.format(sub))
            sub_deriv_dir = op.join(deriv_dir, op.dirname('/'.join(bf_dir_list[idx:])))
            bf_filename = bf_dir_list[-1]           
            in_file = op.join(bf_dir, 'merge/vol0000_xform-00000_merged.nii.gz')
            orig_fn_list = bf_filename.split('_')      
            fn_list = orig_fn_list[:-1]
            fn_list.insert(len(fn_list),'bold.nii.gz')
            fn_list.insert(-1, 'space-native')
            fn_list.insert(-1, 'desc-partialPreproc')            
            os.makedirs(('/home/ttha0011/kg98/Thapa/................./5_ExtractedFiles/sub-{}'.format(sub)), exist_ok=True) 
            out_file = op.join(('/home/ttha0011/kg98/Thapa/.............../5_ExtractedFiles/sub-{}'.format(sub)), '_'.join(fn_list))
            #print(out_file)
            #out_file = op.join(sub_deriv_dir, '_'.join(fn_list))
            copyfile(in_file, out_file)  
            #print(out_file)
            
            # Collect native-to-T1w and T1w-to-MNI transforms
            out_func_dir = op.dirname(out_file)
            #print(out_func_dir)            
            f = op.join(task_dir, 'bold_std_trans_wf', '_key_MNI152NLin2009cAsym/bold_to_std_transform/_inputs.pklz')
            with gzip.open(f, 'rb') as fo:
                #print (fo)
                data = pickle.load(fo)  


## Organise files for Tedana & Transformations

In [6]:
#Step 1: Enter subject ID
subID = ['101', '238']

# subID = ["102", "103", "104", "105", "106", "107", "108", "109", "110", "111", "112", "113", "114", "115", "116", "117", "118", "119", "120",
#          "121", "122", "123", "124", "125", "126", "127", "128", "129", "130", "131", "132", "133", "136", "137", "138", "139", "140", "141", "142",
#          "143",	"201", "202", "204", "205", "206", "207", "208", "209", "210", "211", "212", "213", "214", "215", "216", "217", "218", "219",
#          "220", "221", "222", "223", "224", "225", "226", "227", "228", "229", "230", "231", "232", "233", "234", "235", "236", "237", "238", "239",
#          "240",	"241", "242", "243"]

for sub in subID:
    
    #Step 2: Define path to fMRIPrep folder
    path = ('/projects/...............')    
    
    sub_in_dir = op.join(work_dir + '{0}/fmriprep_wf/single_subject_{0}_wf'.format(sub))

    #Step 4: Define path to anatomical folder. Need the xfm.h5 for T1w_to_MNI transformations
    anatpath = op.join(path + '/data/derivatives/fmriprep/fmriprep/sub-{}/anat'.format(sub))
    
    #Step 5: Define path to functional folder. Need the exhoes and the transformation files
    funcpath = op.join(path +'/data/derivatives/fmriprep/fmriprep/sub-{}/func'.format(sub))

    #Step 6: Define path to bold_hmc_wfpath folder.
    bold_hmc_wfpath = op.join(path + '/scripts/work/{}/fmriprep_wf/single_subject_{}_wf/func_preproc_task_REST_echo_1_wf/bold_hmc_wf/fsl2itk'.format (sub,sub))

    #Step 7: Define path to bold_hmc_wfpath folder.
    affine_wfpath = op.join(path + '/scripts/work/{}/fmriprep_wf/single_subject_{}_wf/func_preproc_task_REST_echo_1_wf/bold_reg_wf/bbreg_wf/fsl2itk_fwd'.format (sub,sub))
    
    #Step 8: Define path to bold_hmc_wfpath folder.
    sdc_wfpath = op.join(path + '/scripts/work/{}/fmriprep_wf/single_subject_{}_wf/func_preproc_task_REST_echo_1_wf/sdc_wf/pepolar_unwarp_wf/cphdr_warp/'.format (sub,sub))

    #Step 9: Define path to where you will copy the extracted echoes, and transformation files
    ExtractedFilesFolder = ('/home/ttha0011/kg98/Thapa/............./5_ExtractedFiles/sub-{}'.format(sub))

    #Step 10: Define the files to copy. These should be the same for each participant. File1 = anat folder; file2 - 7: func files
    file1 = ('sub-{}_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5'.format(sub))
    file2 = ('mat2itk.txt')
    file3 = ('affine.txt'.format(sub))
    file4 = ('_warpfieldQwarp_PLUS_WARP_fixhdr.nii.gz')

    #Step 10: Copy files to the right dirctory i.e., the ConditionFolder. File1 = anat folder; file2 - 7: func files
    shutil.copy(anatpath + '/' + file1, ExtractedFilesFolder)
    shutil.copy(bold_hmc_wfpath + '/' + file2, ExtractedFilesFolder)
    shutil.copy(affine_wfpath + '/' + file3, ExtractedFilesFolder)
    shutil.copy(sdc_wfpath + '/' + file4, ExtractedFilesFolder)
    
    #Step 11: rename files
    hmc_file = op.join(ExtractedFilesFolder + '/mat2itk.txt')
    rename_hmc_file = op.join(ExtractedFilesFolder + '/' + ('sub-{0}_task-REST_from-native_to-reference_mode-image_xfm.txt').format(sub))
    os.rename(hmc_file, rename_hmc_file)
    
    affine_file = op.join(ExtractedFilesFolder + '/affine.txt')
    rename_affine_file = op.join(ExtractedFilesFolder + '/' + ('sub-{0}_task-REST_from-reference_to-T1w_mode-image_xfm.txt').format(sub))
    os.rename(affine_file, rename_affine_file)
    
    sdc_file = op.join(ExtractedFilesFolder + '/_warpfieldQwarp_PLUS_WARP_fixhdr.nii.gz')
    rename_sdc_file = op.join(ExtractedFilesFolder + '/' + ('sub-{0}_task-REST_sdc_warpfieldQwarp.nii.gz').format(sub))
    os.rename(sdc_file, rename_sdc_file)    
    
    #Step 12: Define path to where you will copy the extracted echoes
    os.makedirs((('/home/ttha0011/kg98/Thapa/............/6_Tedana/sub-{}'.format(sub))), exist_ok=True)
    TedanaFolder = ('/home/ttha0011/kg98/Thapa/.........../6_Tedana/sub-{}'.format(sub))
    
    #Step 13: Make input (echoes files) and output (Tedana ouputs) folders in each condition's folder 
    os.makedirs(TedanaFolder + '/Input', exist_ok=True) 
    os.makedirs(TedanaFolder + '/Output', exist_ok=True) 
    
    #Step 14: Input folder
    EchoesFolder = op.join(TedanaFolder + '/' + 'Input')
    
    #Step 15: Define echo names 
    Echo1 = ('sub-{}_task-REST_echo-1_space-native_desc-partialPreproc_bold.nii.gz'.format(sub))
    Echo2 = ('sub-{}_task-REST_echo-2_space-native_desc-partialPreproc_bold.nii.gz'.format(sub))
    Echo3 = ('sub-{}_task-REST_echo-3_space-native_desc-partialPreproc_bold.nii.gz'.format(sub))
    Echo4 = ('sub-{}_task-REST_echo-4_space-native_desc-partialPreproc_bold.nii.gz'.format(sub))

    #Step 16: Copy, and paste each echo in the Tedana folder 
    shutil.copy(ExtractedFilesFolder + '/' + Echo1, EchoesFolder)
    shutil.copy(ExtractedFilesFolder + '/' + Echo2, EchoesFolder)
    shutil.copy(ExtractedFilesFolder + '/' + Echo3, EchoesFolder)
    shutil.copy(ExtractedFilesFolder + '/' + Echo4, EchoesFolder)
    
    #Step 17: Define path to where you will copy the transformation files
    os.makedirs((('/home/ttha0011/kg98/Thapa/WinnieData/7_Transformations/sub-{}'.format(sub))), exist_ok=True)
    TransformationsFolder = ('/home/ttha0011/kg98/Thapa/WinnieData/7_Transformations/sub-{}'.format(sub))
        
    #Step 18: Define transformation files
    Trans1 = ('sub-{}_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5'.format(sub))
    Trans2 = ('sub-{}_task-REST_from-native_to-reference_mode-image_xfm.txt'.format(sub))
    Trans3 = ('sub-{}_task-REST_from-reference_to-T1w_mode-image_xfm.txt'.format(sub))
    Trans4 = ('sub-{}_task-REST_sdc_warpfieldQwarp.nii.gz'.format(sub))
    
    #Step 19: Copy, and paste transformation files to the transformation folder
    shutil.copy(ExtractedFilesFolder + '/' + Trans1, TransformationsFolder)
    shutil.copy(ExtractedFilesFolder + '/' + Trans2, TransformationsFolder)
    shutil.copy(ExtractedFilesFolder + '/' + Trans3, TransformationsFolder)
    shutil.copy(ExtractedFilesFolder + '/' + Trans4, TransformationsFolder)

This section copies the transformations to run ANTs